In [ ]:
from wave_client import WaveClient
import pandas as pd
from utils import (
    load_environment_variables,
)


In [ ]:
# Load environment variables and connect to WAVE backend
RESEARCHER_API_KEY, EXPERIMENTEE_API_KEY, WAVE_BACKEND_URL = load_environment_variables()

print(f"🔍 Connecting to WAVE backend...")
print(f"📡 Backend URL: {WAVE_BACKEND_URL}")
print(f"🔑 Using Researcher API key ending in: ...{RESEARCHER_API_KEY[-4:]}")

client = WaveClient(api_key=RESEARCHER_API_KEY, base_url=WAVE_BACKEND_URL)

In [ ]:
## use this (uncomment it) to see what commands exist 
## (allow the scrollable option at the bottom to see all commands)

# help(client.experiments)
# help(client.experiment_data)

In [ ]:
# Get all experiments to see which one to analyze
experiments = await client.experiments.list(skip=0, limit=200)
print(f"Found {len(experiments)} experiments\n")

# Sort by creation date (most recent first) and show up to 5
sorted_experiments = sorted(experiments, key=lambda x: x.get('created_at', ''), reverse=True)[:5]
print(f"Showing {len(sorted_experiments)} most recent experiments (of {len(experiments)} total)\n")

# Display experiment info
for i, exp in enumerate(experiments, 1):
    print(f"{i}. Experiment UUID: {exp['uuid']}")
    print(f"   Description: {exp.get('description', 'N/A')}")
    print(f"   Type ID: {exp.get('experiment_type_id', 'N/A')}")
    print(f"   Created: {exp.get('created_at', 'N/A')[:10]}")
    print(f"   Tags: {', '.join(exp.get('tags', []))}")
    print("-" * 70)


In [ ]:
# Put the necessary info into the variables below 
selected_experiment_uuid = "aeb293f8-2206-46a9-9d46-b21092526504" 

# Get all data from that experiment
print(f"\n📊 Fetching data for experiment {selected_experiment_uuid}...")
experiment_data_df = await client.experiment_data.get_all_data(
    experiment_id=selected_experiment_uuid
)

print(f"Total records: {len(experiment_data_df)}")
# print(experiment_data_df)


In [ ]:
# Filter and save into .csv file
full_df = (experiment_data_df.query("participant_id != '12345678'")
           .sort_values(by="timestamp",ascending=True))
full_df.to_csv("data/dataframe_full.csv", index = False)

answer_df = full_df.query("trial_category == 'afcexpt'")
answer_df.to_csv("data/dataframe_answer.csv", index = False)



In [ ]:
# Example of how to pull out only a few specific columns

selected_columns = answer_df[["response","response_time","chosen_image","target_object","target_shadow"]]
print(selected_columns)